## Preambule

In [1]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [2]:
hist_startyear = 1990

## Paths

In [3]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2")
path_data = path_main / "Data" / str(hist_startyear)
path_figs = path_main / "Figures"
path_ar6_iso = Path("X:/user/dekkerm/Data/IPCC/AR6_ISO3/")

## Read data files

In [4]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [5]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [6]:
df_countries = pd.read_csv("X:/user/dekkerm/Data/AllCountries.csv")

In [7]:
df_data = pd.read_csv(path_ar6_iso / "AR6_Scenarios_Database_ISO3_v1.1.csv")

In [8]:
df_meta = pd.read_excel(path_ar6_iso / "AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name= "meta_Ch3vetted_withclimate")

## Preprocess

In [9]:
mods = np.array(df_meta[df_meta.Category == "C1"].Model)
scens = np.array(df_meta[df_meta.Category == "C1"].Scenario)
modscens = [mods[i]+'|'+scens[i] for i in range(len(mods))]

In [10]:
mods = np.array(df_data.Model)
scens = np.array(df_data.Scenario)
modscens_data = [mods[i]+'|'+scens[i] for i in range(len(scens))]
modscens_data_uni = np.unique([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_data['ModelScenario'] = modscens_data

In [11]:
df_data_small = df_data[df_data.ModelScenario.isin(modscens)]
df_data_small = df_data_small.reset_index(drop=True)
df_data_small = df_data_small[["ModelScenario", "Region", "Variable"]+['1990', '1995',
       '2000', '2005', '2007', '2008', '2010', '2011', '2012', '2013', '2014',
       '2015', '2018', '2020', '2021', '2022', '2023', '2024', '2025', '2030',
       '2035', '2040', '2045', '2050', '2055', '2060', '2065', '2070', '2075',
       '2080', '2085', '2090', '2095', '2100']]

In [12]:
df__ = df_data_small.melt(id_vars=["ModelScenario", "Region", "Variable"], var_name="Time", value_name="Value")
df__['Time'] = df__['Time'].astype(int)
df__ = df__.set_index(['ModelScenario', 'Region', 'Variable', 'Time'])
xr_data = xr.Dataset.from_dataframe(df__)

In [13]:
xr_data = xr_data.reindex(Time = np.arange(1990, 2101))
xr_data = xr_data.interpolate_na(dim="Time", method="linear")

x:\user\dekkerm\environments\MyEnv\lib\site-packages\xarray\core\missing.py:264: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  if not index.is_monotonic:


## Plot

In [14]:
titles = ["GHG emission reduction in 2030", "Policy costs in 2030", "CCS in 2030", "Primary biomass traded in 2030"]
fig = make_subplots(rows=3, cols=3, subplot_titles=titles,
                    horizontal_spacing = 0.025,
                    specs=[[{"colspan": 3,"rowspan": 2}, None, None], [None, None, None], [{},{}, {}]],
                    column_widths=[0.33, 0.33, 0.33],
                    vertical_spacing=0.05)

cols = ['forestgreen', 'tomato', 'steelblue', 'goldenrod', 'purple', 'grey', 'brown',
        'magenta', 'red', 'darkgrey', 'blue', 'darkgreen']
for reg_i, reg in enumerate(['BRA', 'USA', 'CHN', 'IND', 'IDN', "RUS", "EU", "AUS", "JPN", "COL", "MEX", "SAU"]):
    for v_i in range(4):
        row = [1, 3, 3 ,3][v_i]
        col = [1, 1, 2, 3][v_i]
        if v_i == 1: vals = np.array(xr_data.sel(Time=2030, Region=reg, Variable="Policy Cost|Consumption Loss").Value)
        if v_i == 0: vals = np.array((xr_data.sel(Time=2019, Region=reg, Variable="Emissions|Kyoto Gases").Value - xr_data.sel(Time=2030, Region=reg, Variable="Emissions|Kyoto Gases").Value) /xr_data.sel(Time=2020, Region=reg, Variable="Emissions|Kyoto Gases").Value)
        if v_i == 2: vals = np.array(xr_data.sel(Time=2030, Region=reg, Variable="Carbon Sequestration|CCS").Value)
        if v_i == 3: vals = np.array(xr_data.sel(Time=2030, Region=reg, Variable="Trade|Primary Energy|Biomass|Volume").Value)
        fig.add_trace(
            go.Box(y=vals,
                name=reg,
                text=[reg+"<br>"+titles[v_i]+": "+str(np.round(vals[i],1))+"<br>"+modscens[i] for i in range(len(vals))],
                fillcolor=cols[reg_i],
                line_color='black', marker=dict(color='black'),
                boxpoints='all', jitter=0.5, pointpos=-1.8, whiskerwidth=0.2, marker_size=3, line_width=0.5, showlegend=False, hovertemplate='%{text}'),
                row=row, col=col)
        if v_i == 0:
            base = float(xr_total.NDC_l.sel(ISO=reg, Time=2019)) #np.nanmean(xr_data.sel(Time=2020, Region=reg, Variable="Emissions|Kyoto Gases").Value)/1e3
            budget_gf = float(xr_budgets_scenario.sel(Region=reg, Time=2030, Variable="GF", Ccat="C1").Value)
            budget_pc = float(xr_budgets_scenario.sel(Region=reg, Time=2030, Variable="PC", Ccat="C1").Value)
            budget_pcc = float(xr_budgets_scenario.sel(Region=reg, Time=2030, Variable="PCC", Ccat="C1").Value)
            ndc = float(xr_total.NDC_l.sel(ISO=reg, Time=2030))
            fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_gf) / base], text="GF", textfont = {'family': "Arial black", "size": 13, "color": "black"}, marker={'symbol': "circle", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
            fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_pc) / base], text="PC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "square", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
            fig.add_trace(go.Scatter(x=[reg], y=[(base - budget_pcc) / base], text="PCC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "diamond", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)
            fig.add_trace(go.Scatter(x=[reg], y=[(base - ndc) / base], text="NDC", textfont = {'family': "Arial black", "size": 13, "color": "black"},  marker={'symbol': "triangle-up", "color": cols[reg_i], "size": 25, "line": {"width": 0.2, "color": 'black'}, "colorscale": "Spectral",}, mode='markers+text', showlegend=False), row=1, col=1)

fig.update_layout(height=1000, width = 1700)
fig.show()

In [15]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.svg")
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.png", scale=3)

import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_costopt/Figure_costopt.html', 'a') as f:
    #f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))